# Using the Flea3 camera in OpenCV

This demo show you how to use the [Flea3](https://www.flir.co.uk/products/flea3-usb3/) camera from Point Grey (now distributed by [Flir](https://www.flir.co.uk/)). I tested the code in GNU/Linux. There is no reason why it should not work with MS Windows. The code below 

1. opens the camera, 
2. retrieve the frames one by one, 
3. converts them to an [OpenCV Mat](https://docs.opencv.org/master/d3/d63/classcv_1_1Mat.html), 
4. displays them in a window, and
5. save them in a AVI video file.

## Compiler parameters

Set my Jypyter environment for the use of [OpenCV](https://opencv.org/) in a [C++ notebook](https://github.com/jupyter-xeus/xeus-cling). 
You don't need this line when you write your own C++ programs. 
I need it to set my [interactive compiler (Cling)](https://root.cern/cling/). 
For your own program, use [CMake](https://cmake.org/).

In [ ]:
#include "includeLibraries.h"

## Header inclusion for C++

In [ ]:
#include <iostream> // For cout and cerr
#include <sstream> // For formatting error messages
#include <memory> // For smart pointer
#include <stdexcept> // For runtime exceptions
#include <opencv2/opencv.hpp> // For OpenCV
#include <FlyCapture2.h> // For the Flea3 camera

## Add the namespaces

In [ ]:
using namespace FlyCapture2;

In [ ]:
using namespace cv;

In [ ]:
using namespace std;

## Open the camera

In [ ]:
//-----------------------------------
PGRGuid getFlea3Id(int aCameraId = 0)
//-----------------------------------
{
    PGRGuid guid;
    BusManager bus_manager;
    unsigned int number_of_cameras;

    // Retrieve the number of cameras
    FlyCapture2::Error error_status = bus_manager.GetNumOfCameras(&number_of_cameras);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // The camera exists
    if (number_of_cameras > aCameraId)
    {
        error_status = bus_manager.GetCameraFromIndex(aCameraId, &guid);
        if (error_status != PGRERROR_OK)
        {
            throw runtime_error(error_status.GetDescription());
        }
    }
    // The camera does not exist
    else
    {
        stringstream error_message;
        error_message << "FATAL ERROR" << endl;
        error_message << "Number of cameras detected: " << number_of_cameras << endl;
        error_message << "Camera ID " << aCameraId << " does not exist" << endl;
        error_message << "The program will terminate.";
        throw runtime_error(error_message.str());
    }

    return guid;
}

In [ ]:
//------------------------------------------------------
void openFlea3Camera(Camera& aCamera, int aCameraId = 0)
//------------------------------------------------------
{
    // Get the ID
    PGRGuid guid = getFlea3Id(aCameraId);

    // Connect to a camera
    FlyCapture2::Error error_status = aCamera.Connect(&guid);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Get the camera configuration
    FC2Config config;
    error_status = aCamera.GetConfiguration(&config);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Set the number of driver buffers used to 10.
    config.numBuffers = 10;

    // Set the camera configuration
    error_status = aCamera.SetConfiguration(&config);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Start capturing images
    error_status = aCamera.StartCapture();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Read the first frame and throw it away
    // (the data is rubbish)
    Image rawImage;
    error_status = aCamera.RetrieveBuffer(&rawImage);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
}

In [ ]:
//--------------------------------------
void releaseFlea3Camera(Camera& aCamera)
//--------------------------------------
{
    // Stop capturing images
    FlyCapture2::Error error_status = aCamera.StopCapture();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Disconnect the camera
    error_status = aCamera.Disconnect();
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }
}

In [ ]:
//---------------------------
Mat getFrame(Camera& aCamera)
//---------------------------
{
    // Retrieve an image
    Image rawImage;
    FlyCapture2::Error error_status = aCamera.RetrieveBuffer(&rawImage);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Convert the raw image
    Image convertedImage;
    error_status = rawImage.Convert(PIXEL_FORMAT_MONO8, &convertedImage);
    if (error_status != PGRERROR_OK)
    {
        throw runtime_error(error_status.GetDescription());
    }

    // Convert to OpenCV
    Mat opencv_image(convertedImage.GetRows(), convertedImage.GetCols(), CV_8UC1);
    for (int row = 0; row < convertedImage.GetRows(); ++row)
    {
        for (int col = 0; col < convertedImage.GetCols(); ++col)
        {
            opencv_image.at<unsigned char>(Point(col, row)) = *convertedImage(row, col);

        }
    }

    return opencv_image;
}

Open the default camera (ID=0)

In [20]:
Camera camera;
openFlea3Camera(camera, 0);

Standard Exception: FATAL ERROR
Number of cameras detected: 0
Camera ID 0 does not exist
The program will terminate.

Grab a new frame and open the video writer

In [ ]:
Mat frame = getFrame(camera);
VideoWriter video_output("../output.avi", VideoWriter::fourcc('M','J','P','G'), 25, Size(frame.cols,frame.rows));

## Display the images from the camera

We create a window to display the images from the webcam

In [ ]:
namedWindow("Webcam", WINDOW_GUI_EXPANDED); // Create a window

We display the images in a loop

In [ ]:
while (waitKey(25) != 27) // Exit when pressing <ESC>
{
    // Grab a new frame
    Mat frame = getFrame(camera);

    // Make sure everything went well
    if (frame.empty())
    {
        releaseFlea3Camera(camera);
        throw runtime_error("OpenCV cannot grab a new frame from the camera, the program will terminate");
    }

    // Save the frame in the output file
    video_output << frame;

    // Display the image
    imshow("Webcam", frame);
}

We are now done with the video output, stop it

In [ ]:
video_output.release();

Release the camera

In [23]:
releaseFlea3Camera(camera);

Standard Exception: Generation count is incorrect.

We don't need the window any more, destroy it

In [ ]:
destroyAllWindows();